# Explore the Churchlandlab pipeline

DataJoint provides tools to explore the existing pipelines.In this notebook, we would like to cover the following functionalities:

>* `dj.list_schemas()`
>* `dj.Diagram()`
>* `describe()` and `heading`
>* Queries
>* `fetch()`
>* `delete()`

# List all the schemas you have access to, using `dj.list_schemas()`

In [ ]:
import datajoint as dj
dj.list_schemas()

# Access the schemas

Each of the schemas starting with `churchland_common` is created and could be accessed using `churchland_pipeline_python` 

In [ ]:
from churchland_pipeline_python import lab, equipment, reference, action, acquisition, processing

The reason that these modules could be used to access the schemas because there is a schema object in these modules.

In [ ]:
lab.schema

In [ ]:
action.schema

Table classes are also defined in the modules

In [ ]:
lab.Monkey()

# Use `dj.Diagram()` to overview the tables and dependencies

In [ ]:
# schema object as input
dj.Diagram(reference.schema)

In [ ]:
# module that contains schema object as input
dj.Diagram(reference)

In [ ]:
# table as input
dj.Diagram(reference.Muscle)

In [ ]:
dj.Diagram(reference.Muscle) + dj.Diagram(reference.BrainRegion) + dj.Diagram(action)

In [ ]:
dj.Diagram(lab)

In [ ]:
# diagram of a schema sophisticated dependencies
dj.Diagram(equipment)

In [ ]:
dj.Diagram(acquisition)

In [ ]:
dj.Diagram(processing)

Here are some notes on what these shapes, colors and lines mean:

**Table tiers**:  
Manual table: green box, metadata that needs to be inserted manually, e.g. Session.  
Lookup table: gray box, some general facts, or parameters, e.g. BrainRegion.  
Imported table: blue oval, autoprocessed tables that depend on external data.   
Computed table: red circle, autoprocessed tables that only depend on data inside database.  
Part table: plain text, extension tables.  


**Dependencies**:  
One-to-one primary: thick solid line  
One-to-many primary: thin solid line  
Secondary foreign key reference: dashed line  
Renamed secondary foreign key references: orange dot

# Check table definitions using `describe()`, `heading`, and check the contents with the preview

In [ ]:
# show definition with dependencies
acquisition.Session.describe();

In [ ]:
lab.Monkey.describe();

In [ ]:
lab.Rig.describe();

In [ ]:
acquisition.Task.describe();

In [ ]:
acquisition.Session.heading

In [ ]:
# preview contents
acquisition.Session()

In [ ]:
acquisition.EphysRecording.describe();

In [ ]:
acquisition.EphysRecording()

# Use `dj.Diagram` to check upstream and downstream dependencies

In [ ]:
dj.Diagram(acquisition.Session) - 1

In [ ]:
acquisition.EphysRecording.describe();

In [ ]:
dj.Diagram(acquisition.Session) + 1 - 1

In [ ]:
dj.Diagram(acquisition) + dj.Diagram(processing) - (dj.Diagram(processing.Filter) + 1)

# Queries


In [ ]:
# a table preview is the simplest query, this returns a query object
acquisition.Session()

In [ ]:
q = acquisition.Session()
q

## Restrictions (`&`) - filter data with certain conditions

The **restriction** operation, `&`, let's you specify the criteria to narrow down the table on the left.

### Exact match

In [ ]:
# string as a restrictor
acquisition.Session & 'monkey="Cousteau"'

In [ ]:
# dict as a restrictor
acquisition.Session & {'monkey': 'Cousteau'}

### Inequality

In [ ]:
acquisition.Session & 'session_date > "2020-01-01"'

In [ ]:
acquisition.Session & 'session_date between "2019-01-01" and "2020-01-01"'

In [ ]:
acquisition.Session & 'session_date != "2020-01-10"'

### 'OR' logic

In [ ]:
# or in the string (most mysql syntax will work inside the string)
acquisition.Session & 'session_date="2020-01-06" or session_date="2019-11-01"'

In [ ]:
acquisition.Session & 'session_date in ("2020-01-06", "2019-11-01")'

In [ ]:
# restrict with a list
acquisition.Session & ['session_date="2020-01-06"', {'session_date': '2019-11-01'}]

### Restriction one table with another

In [ ]:
# sessions with ephys recordings
acquisition.Session & acquisition.EphysRecording

In [ ]:
acquisition.EphysRecording()

### Negative restriction `-`

In [ ]:
# sessions without ephys recordings
acquisition.Session - acquisition.EphysRecording

### Combining restrictions

In [ ]:
# put multiple restrictions in one dictionary
acquisition.Session & {'monkey': 'Cousteau', 'session_date': '2020-01-10'}

All the above queries could be combined 

In [ ]:
# sessions with behavior and ephys recordings after '2020-01-01'
acquisition.Session & acquisition.BehaviorRecording & acquisition.EphysRecording & 'session_date > "2020-01-01"'

In [ ]:
# sessions with behavior recording and without ephys recordings
acquisition.Session & acquisition.BehaviorRecording - acquisition.EphysRecording

In [ ]:
# neurons recorded with Neuropixels
processing.Neuron & (acquisition.BrainChannelGroup & {'electrode_array_model': 'Neuropixels'})

In [ ]:
# sessions with sorted neurons and motor units
acquisition.Session & processing.Neuron & processing.MotorUnit

In [ ]:
# neurons from sessions with motor units
processing.Neuron & processing.MotorUnit

## Joining (*)  - bring fields from different tables together

Sometimes you want to see information from multiple tables combined together to be viewed (and queried!) simultaneously. You can do this using the join `*` operator.

Behavior of join:

1. match the common field(s) of the primary keys in the two tables
2. do a combination of the non-matched part of the primary key
3. listing out the secondary attributes for each combination
4. if two tables have secondary attributes that share a same name, it will throw an error. To join, we need to rename that attribute for at least one of the tables.

In [ ]:
# behavior and ephys recordings
acquisition.BehaviorRecording * acquisition.EphysRecording

In [ ]:
# emg channel group
acquisition.EmgChannelGroup()

In [ ]:
# muscle list
reference.Muscle()

In [ ]:
# join emg channel group with full muscle name and head
q = acquisition.EmgChannelGroup * reference.Muscle
q

In [ ]:
# restrict to only `AntDel`, and recorded after '2019-01-01'
q & 'muscle="deltoid"' & 'session_date > "2019-01-01"'

## Projection `.proj()`
Besides restriction (`&`) and join (`*`) operations, DataJoint offers another type of operation: projection (`.proj()`). Projection is used to 
1. select attributes (columns) from a table, 
2. rename attributes,
3. create new calculated attributes. 

### Focus on attributes of interest

In [ ]:
acquisition.Session()

In [ ]:
acquisition.Session.proj()

In [ ]:
acquisition.Session.proj('rig', 'task')

### Rename attributes

In [ ]:
lab.Monkey.proj(date_of_birth='dob')

### Perform simple computations

In [ ]:
acquisition.EphysRecording()

In [ ]:
acquisition.EphysRecording.File()

In [ ]:
ephys_file = acquisition.EphysRecording * acquisition.EphysRecording.File & {'session_date': '2019-12-16'}
ephys_file

In [ ]:
# create full ephys data path
ephys_file.proj(ephys_file_path='CONCAT(ephys_recording_path, ephys_file_path, ephys_file_name, ".", ephys_file_extension)')

In [ ]:
# monkeys age in years when performing each session
(lab.Monkey * acquisition.Session).proj(age='datediff(session_date, dob)/365')

In [ ]:
# keep the original attributes while computing new ones:
(lab.Monkey * acquisition.Session).proj(..., age='datediff(session_date, dob)/365')

# Aggregation `aggr()`: computation across different entries

Computations with `proj()` were only performed among attributes within each single entry, while `aggr()` performs computation across different entries. Here is an example.

In [ ]:
# How many neurons were recorded in each EphysRecording?
acquisition.EphysRecording.aggr(processing.Neuron, n_neurons='count(neuron_id)')

For more functions that could be used in proj(), aggr() and string of restriction, refer to the [mysql documentation](https://dev.mysql.com/doc/refman/8.0/en/sql-function-reference.html), or [this doc](https://www.w3schools.com/sql/sql_ref_mysql.asp).
    

# Universal sets

Universal sets (dj.U) defines the set of all possible entries.

Combined with `&`, `*`, and `aggr()`, it can be used to flexibly create new entity types not stored in existing tables.

In [ ]:
dj.U()

In [ ]:
acquisition.EmgChannelGroup()

In [ ]:
# Unique muscle in all emg recording
dj.U('muscle_abbr') & acquisition.EmgChannelGroup()

The following examples requires installation of [datajoint-pacman](https://github.com/ChurchlandLabCUMC/datajoint-pacman)

In [ ]:
from pacman_pipeline_python import pacman_acquisition

In [ ]:
# Trial table of pacman
pacman_acquisition.Behavior.Trial()

In [ ]:
# all unique save tags (2020-01-06)
trial_table = pacman_acquisition.Behavior.Trial & {'session_date': '2020-01-06'}
dj.U('save_tag') & trial_table

In [ ]:
# trial counts per save tag
dj.U('save_tag').aggr(trial_table, count='count(*)')

In [ ]:
# success rate per save tag
dj.U('save_tag').aggr(trial_table, success_rate='avg(successful_trial)')

In [ ]:
# project a random ID to each trial
dj.U('trial').aggr(trial_table, rnd_id='rand()')

In [ ]:
# fetch a random trial key
(trial_table * dj.U('trial').aggr(trial_table, rnd_id='rand()')).fetch('KEY', limit=1, order_by='rnd_id')

# Fetch data

So far, we just queried the tables, but we haven't got any data from the server in python data types. The method `fetch()` will allow us to do that from any table or query results.

## Fetch one or multiple entries: `fetch()`

In [ ]:
# fetch all the monkey data
lab.Monkey.fetch()

In [ ]:
# fetch from a query
(lab.Monkey & 'dob > "2005-01-01"').fetch()

In [ ]:
# as a list of dictionary
(lab.Monkey & 'dob > "2005-01-01"').fetch(as_dict=True)

In [ ]:
# as a pandas dataframe
(lab.Monkey & 'dob > "2005-01-01"').fetch(format='frame')

In [ ]:
# only fetch the primary key
(lab.Monkey & 'dob > "2005-01-01"').fetch('KEY')

In [ ]:
# fetch some fields
dob, sex = (lab.Monkey & 'dob > "2005-01-01"').fetch('dob', 'sex')
dob

In [ ]:
sex

In [ ]:
# fetch fields as list of dict
(lab.Monkey & 'dob > "2005-01-01"').fetch('KEY', 'dob', 'sex', as_dict=True)

In [ ]:
# fetch only limit to a few entries
acquisition.Session.fetch(limit=3, as_dict=True)

In [ ]:
acquisition.EmgChannelGroup()

In [ ]:
# get data in order by some fields
acquisition.EmgChannelGroup.fetch('emg_channel_group_id', as_dict=True, order_by='emg_channel_group_id')

In [ ]:
acquisition.EmgChannelGroup.fetch('emg_channel_group_id', as_dict=True)

## Fetch data from only one entry: `fetch1()`

In [ ]:
# fetch entire entry as a dict
(lab.Monkey & {'monkey': 'Alex'}).fetch1()

In [ ]:
# it returns a list of dict if using fetch()
(lab.Monkey & {'monkey': 'Alex'}).fetch(as_dict=True)

In [ ]:
# fetch a field
(lab.Monkey & {'monkey': 'Alex'}).fetch1('dob')

In [ ]:
# fetch multiple fields
(lab.Monkey & {'monkey': 'Alex'}).fetch1('dob', 'sex')

In [ ]:
# fetch multiple fields
dob, sex = (lab.Monkey & {'monkey': 'Alex'}).fetch1('dob', 'sex')
print(dob, sex)

In [ ]:
# fetch primary key
(lab.Monkey & {'monkey': 'Alex'}).fetch1('KEY')